In [ ]:
#@title Initialize environment { vertical-output: true, display-mode: "form" }
import os
import subprocess
import random
import time

import glob

try:
    import fiona
except ImportError:
    print('fiona package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'fiona'])
    import fiona

try:
    import rasterio.merge
except ImportError:
    print('rasterio package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'rasterio'])
    import rasterio.merge




fiona package not installed. Installing ...
rasterio package not installed. Installing ...


In [ ]:
#@title Initialize Google Drive  environments { vertical-output: true, display-mode: "form" }
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
#@title Initialize GEE environments { vertical-output: true, display-mode: "form" }
import ee
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize()

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
    import geemap

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=myHJ3wSY8c4ZX2dyWBQarObzIxvQI3dnmdtmJ17wWEQ&tc=NoX5PsteDUOBSQgVD4fyphd-5HF0RdDUx5DyesRHAWk&cc=ZUzXtQOnY-kmo5zlQ2I2Qp0mfPTh48vOKH4Sxw_MXoA

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfgeXvvP062xc2tloCXlBAxqDZ3lRKb4rwNEgwF943cLpFYQpfR3hoI7mAI

Successfully saved authorization token.
geemap package not installed. Installing ...


In [ ]:
!rm my_vector.gpkg

In [ ]:
#@title Import vector layer wich contained border areas for export  { vertical-output: true, display-mode: "form" }
#@markdown Path on googledrive and file name "gpkg"
folder_path = "GeoData/InstaMaps/Portugal/" #@param {type:"string"}
vetor_zones_file_name =  "srtm_adm_2_simple"#@param {type:"string"}

vetor_zones_file = vetor_zones_file_name+'.gpkg' # grid file

subprocess.check_call(['cp', '/content/gdrive/MyDrive/'+folder_path+vetor_zones_file, '.'])

!ls

gdrive				   srtm_adm_1_simple.gpkg
portugal_adm1_landcover_stats.csv  srtm_adm_2_simple.gpkg
sample_data


In [ ]:
#@title Output settings  { vertical-output: true, display-mode: "form" }
#@markdown 1) Path on googledrive;

#@markdown 2) export CSV filename

#@markdown 3) Statistics_type 'SUM' or 'PERCENTAGE';

#@markdown 4) denominator can be used to convert square meters to other areal units, such as square kilimeters

#@markdown 5) decimal_places - number ;


export_path = "GeoData/InstaMaps/Portugal" #@param {type:"string"}
export_filename = "portugal_adm2_landcover_stats.csv" #@param {type:"string"}
statistics_type = "SUM" #@param {type:"string"}
denominator = 1000000 #@param {type:"integer"}
decimal_places = 2 #@param {type:"integer"}
#scale =  10#@param {type:"integer"}
#crs_epsg = "3857" #@param {type:"string"}


In [ ]:
#@title Convert grid to ee.FeatureCollection  { vertical-output: true, display-mode: "form" }
polygon_features = []
with fiona.open(vetor_zones_file) as layer:
    for feature in layer:
        polygon_features.append(ee.Feature(feature))
AOI = ee.FeatureCollection(polygon_features)


In [ ]:
#@title Import <b>ESA WorldCover 10m v200</b>  { vertical-output: true, display-mode: "form" }
#@markdown https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v200

#@markdown 10,Tree cover,006400

#@markdown 20,Shrubland,ffbb22

#@markdown 30,Grassland,ffff4c

#@markdown 40,Cropland,f096ff

#@markdown 50,Built-up,fa0000

#@markdown 60,Bare / sparse vegetation,b4b4b4

#@markdown 70,Snow and ice,f0f0f0

#@markdown 80,Permanent water bodies,0064c8

#@markdown 90,Herbaceous wetland,0096a0

#@markdown 95,Mangroves,00cf75

#@markdown 100,Moss and lichen,fae6a0


landcover = ee.ImageCollection('ESA/WorldCover/v200').first();
landcover_visualization = {'bands': ['Map'],};



In [ ]:
#@title Create map
Map = []

Map = geemap.Map(center=(39.633, -8.197), zoom=6)

Map.addLayer(landcover, landcover_visualization, 'Landcover', True, 1)
Map.addLayer(AOI, {}, 'Zones', True, 1)

Map


Map(center=[39.633, -8.197], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(child…

In [ ]:
Map.add_legend(builtin_legend='ESA_WorldCover')

In [ ]:
#@title Calculate land cover compostion of each polygons

out_dir = os.path.join('/content/gdrive/MyDrive', export_path)
landcover_stats = os.path.join('/content', export_filename)

if not os.path.exists(out_dir):
    os.makedirs(out_dir)





In [ ]:
%%time
geemap.zonal_statistics_by_group(
    landcover,
    AOI,
    landcover_stats,
    statistics_type = statistics_type,
    denominator = denominator,
    decimal_places = decimal_places,
)

Computing ... 
Generating URL ...
Please wait ...
Data downloaded to /content/portugal_adm2_landcover_stats.csv
CPU times: user 12.2 s, sys: 334 ms, total: 12.6 s
Wall time: 1min 20s


In [ ]:
geemap.create_download_link(landcover_stats)

In [ ]:
%%time
subprocess.check_call(['cp', landcover_stats, out_dir])


CPU times: user 2.64 ms, sys: 9.38 ms, total: 12 ms
Wall time: 31.6 ms


0